In [ ]:
#Import functions------------------------------------------------------------------------------------------------------
from my_functions import*

#Open ROOT file with uproot--------------------------------------------------------------------------------------------
file= uproot.open("/home/ruben/Escritorio/BachelorThesisRuben/Data/Muon_trees.root") 

#Get trees-------------------------------------------------------------------------------------------------------------
MuonTree_ZeroBias=file["MuonTree_ZeroBias;1"]
MuonTree_Zmumu=file["MuonTree_Zmumu;1"]


nmin1=0
nmax1=5000

Zmumu_pt=quality_selector_with_empty(MuonTree_Zmumu["muon_quality"].array(),MuonTree_Zmumu["muon_pt"].array(),0)[nmin1:nmax1]
Zmumu_eta=quality_selector_with_empty(MuonTree_Zmumu["muon_quality"].array(),MuonTree_Zmumu["muon_eta"].array(),0)[nmin1:nmax1]
Zmumu_phi=quality_selector_with_empty(MuonTree_Zmumu["muon_quality"].array(),MuonTree_Zmumu["muon_phi"].array(),0)[nmin1:nmax1]
#Select the Z peak pairs
Zmumu_pt, Zmumu_eta, Zmumu_phi= get_all_Z_peak_pairs(Zmumu_pt,Zmumu_eta,Zmumu_phi)

dr_min=0.1
dr_max=0.6
Zmumu_iso=muon_isolation_all_events(MuonTree_Zmumu, Zmumu_eta, Zmumu_phi, dr_min, dr_max, [nmin1, nmax1])
Zmumu_ratio=ak.flatten(Zmumu_iso)/ak.flatten(Zmumu_pt)

Zmumu_iso=ak.flatten(Zmumu_iso)
Zmumu_iso=Zmumu_iso[~np.isnan(Zmumu_iso)]
l1=len(Zmumu_iso)
l3=len(Zmumu_ratio)


In [ ]:
datasets=1000
e_cut= 14000
ZeroBias_pt_total=[]
ZeroBias_eta_total=[]
ZeroBias_phi_total=[]
iso_total=[]
ratio_total=[]
total_events=0

# Define  directory
base_path = "/home/ruben/Escritorio/BachelorThesisRuben/Data/NewData/"
# List directories 0, 1, 2, ...
folders = [str(f) for f in range(datasets)]
# Loop through the folders
i=0
for folder in tqdm(folders, desc="Loading data...", leave=False):
    file_path = os.path.join(base_path, folder, 'l1calo_hist.root')
    # Check if the file exists before attempting to open it
    if os.path.exists(file_path):
        # Open the ROOT file
        file = uproot.open(file_path)
        #Since datasets 998 and 999 are different, this "if" ensures they are loaded correctly
        if i == 999:
            Tree="MuonTree_ZeroBias;1"
            MuonTree_ZeroBias=file[Tree]
        else:
            Tree="MuonTree;1"
            MuonTree_ZeroBias=file[Tree]
        
        ###############################################################################################################################3
        #Select ZeroBias
        total_events=total_events+len(MuonTree_ZeroBias["muon_pt"].array())
        ZeroBias_pt=energy_cut_with_empty(MuonTree_ZeroBias["muon_pt"].array(), MuonTree_ZeroBias["muon_pt"].array(), e_cut)
        ZeroBias_eta=energy_cut_with_empty(MuonTree_ZeroBias["muon_pt"].array(), MuonTree_ZeroBias["muon_eta"].array(), e_cut)
        ZeroBias_phi=energy_cut_with_empty(MuonTree_ZeroBias["muon_pt"].array(), MuonTree_ZeroBias["muon_phi"].array(), e_cut)

        Zbl1_pt = MuonTree_ZeroBias["LVL1Muon_et"].array() * 1000
        Zbl1_eta = MuonTree_ZeroBias["LVL1Muon_eta"].array()
        Zbl1_phi = MuonTree_ZeroBias["LVL1Muon_phi"].array()

        #Select LVL1 ZeroBias
        mask1 = Zbl1_pt >= e_cut

        Zbl1_pt=Zbl1_pt[mask1]
        Zbl1_eta=Zbl1_eta[mask1]
        Zbl1_phi=Zbl1_phi[mask1]
        #Compute matchings
        mask2 = offline_LVL1_matcher(ZeroBias_eta, ZeroBias_phi, Zbl1_eta, Zbl1_phi)
        #Apply them
        ZeroBias_pt=ZeroBias_pt[mask2]
        ZeroBias_eta=ZeroBias_eta[mask2]
        ZeroBias_phi=ZeroBias_phi[mask2]
        #Compute isolations 
        iso=muon_isolation_all_events(MuonTree_ZeroBias, ZeroBias_eta, ZeroBias_phi, dr_min, dr_max, [0, len(ZeroBias_pt)])
        ratio=ak.flatten(iso)/ak.flatten(ZeroBias_pt)
        #Concatenate with other data
        ZeroBias_pt_total=ak.concatenate([ZeroBias_pt_total,ZeroBias_pt[ak.num(ZeroBias_pt) > 0]])
        ZeroBias_eta_total=ak.concatenate([ZeroBias_eta_total,ZeroBias_eta[ak.num(ZeroBias_pt) > 0]])
        ZeroBias_phi_total=ak.concatenate([ZeroBias_phi_total,ZeroBias_phi[ak.num(ZeroBias_pt) > 0]])

        iso_total=ak.concatenate([iso_total, iso])
        ratio_total=ak.concatenate([ratio_total, ratio])

    i=i+1
#Remove nan and empty
iso_total=iso_total[~np.isnan(iso_total)]
ZeroBias_iso=iso_total[ak.num(iso_total) > 0]
ZeroBias_ratio=ratio_total[~np.isnan(ratio_total)]
print("Number of events scanned", total_events)
print("Post-selected ZeroBias length", len(ZeroBias_pt_total))
print("Total Isolation length", len(ZeroBias_iso))

In [ ]:
datasets=1000
e_cut= 14000
ZeroBias_pt_total=[]
ZeroBias_eta_total=[]
ZeroBias_phi_total=[]
iso_total=[]
ratio_total=[]
num_events=0
batch_size=1000
start_event=0
# Define  directory
base_path = "/home/ruben/Escritorio/BachelorThesisRuben/Data/NewData/"
# List directories 0, 1, 2, ...
folders = [str(f) for f in range(datasets)]
# Loop through the folders
i=0
for folder in tqdm(folders, desc="Loading data...", leave=False):
    file_path = os.path.join(base_path, folder, 'l1calo_hist.root')
    # Check if the file exists before attempting to open it
    if os.path.exists(file_path):
        # Open the ROOT file
        file = uproot.open(file_path)
        #Since datasets 998 and 999 are different, this "if" ensures they are loaded correctly
        if i == 999:
            Tree="MuonTree_ZeroBias;1"
            MuonTree_ZeroBias=file[Tree]
        else:
            Tree="MuonTree;1"
            MuonTree_ZeroBias=file[Tree]
        
        ###############################################################################################################################3
        #Select ZeroBias
        num_events=num_events+len(MuonTree_ZeroBias["muon_pt"].array())
        ZeroBias_pt=energy_cut_with_empty(MuonTree_ZeroBias["muon_pt"].array(), MuonTree_ZeroBias["muon_pt"].array(), e_cut)
        ZeroBias_eta=energy_cut_with_empty(MuonTree_ZeroBias["muon_pt"].array(), MuonTree_ZeroBias["muon_eta"].array(), e_cut)
        ZeroBias_phi=energy_cut_with_empty(MuonTree_ZeroBias["muon_pt"].array(), MuonTree_ZeroBias["muon_phi"].array(), e_cut)

        Zbl1_pt = MuonTree_ZeroBias["LVL1Muon_et"].array() * 1000
        Zbl1_eta = MuonTree_ZeroBias["LVL1Muon_eta"].array()
        Zbl1_phi = MuonTree_ZeroBias["LVL1Muon_phi"].array()

        #Select LVL1 ZeroBias
        mask1 = Zbl1_pt >= e_cut

        Zbl1_pt=Zbl1_pt[mask1]
        Zbl1_eta=Zbl1_eta[mask1]
        Zbl1_phi=Zbl1_phi[mask1]
        #Compute matchings
        mask2 = offline_LVL1_matcher(ZeroBias_eta, ZeroBias_phi, Zbl1_eta, Zbl1_phi)
        #Apply them
        ZeroBias_pt=ZeroBias_pt[mask2]
        ZeroBias_eta=ZeroBias_eta[mask2]
        ZeroBias_phi=ZeroBias_phi[mask2]

        #Apply masks to jTowers
        jTower = MuonTree_ZeroBias.arrays(["jTower_eta", "jTower_phi"], entry_start=0, entry_stop=1)
        jTower_eta_event = jTower["jTower_eta"][0]
        jTower_phi_event = jTower["jTower_phi"][0]

        for batch_start in tqdm(range(0, total_events, batch_size), desc="muon_isolation_all_events: Computing muon isolation", leave=False):
            batch_stop = min(batch_start + batch_size, total_events)
            # Load jTower data only for the current batch
            jTower = MuonTree_ZeroBias.arrays(["jTower_et_MeV", "jTower_eta", "jTower_phi"],
                                               entry_start=start_event + batch_start, entry_stop=start_event + batch_stop)
            #assign et
            jTower_et_batch  = jTower["jTower_et_MeV"]
            jTower_eta_batch  = jTower["jTower_eta"]
            jTower_phi_batch  = jTower["jTower_phi"]
        #Concatenate with other data
        ZeroBias_pt_total=ak.concatenate([ZeroBias_pt_total,ZeroBias_pt[ak.num(ZeroBias_pt) > 0]])
        ZeroBias_eta_total=ak.concatenate([ZeroBias_eta_total,ZeroBias_eta[ak.num(ZeroBias_pt) > 0]])
        ZeroBias_phi_total=ak.concatenate([ZeroBias_phi_total,ZeroBias_phi[ak.num(ZeroBias_pt) > 0]])

        iso_total=ak.concatenate([iso_total, iso])
        ratio_total=ak.concatenate([ratio_total, ratio])

    i=i+1
#Remove nan and empty
iso_total=iso_total[~np.isnan(iso_total)]
ZeroBias_iso=iso_total[ak.num(iso_total) > 0]
ZeroBias_ratio=ratio_total[~np.isnan(ratio_total)]
print("Number of events scanned", num_events)
print("Post-selected ZeroBias length", len(ZeroBias_pt_total))
print("Total Isolation length", len(ZeroBias_iso))

In [ ]:
import ROOT

# Function to filter events (you can modify the condition as per your needs)
def event_filter(event):
    # Here, you can define your custom condition to select events
    # For example: select events where variable "energy" is above 100
    energy = event.GetLeaf("energy").GetValue()  # Assuming 'energy' is a branch in your tree
    if energy > 100:  # Simple condition to filter events
        return True
    return False

# Create a new ROOT file to store selected events
output_file = ROOT.TFile("combined_events.root", "RECREATE")

# Create a tree in the output file
output_tree = ROOT.TTree("Events", "Selected Events Tree")

# Define branches that will be copied to the new file (assuming the event has these variables)
event = ROOT.TTree()
event.Branch("energy", ROOT.Double(0), "energy/D")  # Adjust the branch definition as per your data

# List of input ROOT files (add your file paths here)
input_files = ["file1.root", "file2.root", "file3.root"]  # Replace with your file names

# Loop through the ROOT files
for file_name in input_files:
    input_file = ROOT.TFile(file_name, "READ")
    tree = input_file.Get("Events")  # Replace "Events" with your tree name if different

    # Loop through all events in the tree
    for entry in tree:
        if event_filter(tree):  # Apply the event filter condition
            # You can extract and process branches here as needed
            energy = tree.energy  # Assuming there's an energy branch
            event.SetBranchAddress("energy", energy)  # Set address for your variables
            
            # Fill the tree in the output file with the selected event
            output_tree.Fill()

    # Close the input file
    input_file.Close()

# Write the output tree to the new ROOT file
output_file.Write()

# Close the output file
output_file.Close()

print("Selected events have been written to 'combined_events.root'.")
